- 앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.
- 이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").
- 항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.
- 메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

# 메모리 로드 함수
def load_memory(_):
    return memory.load_memory_variables({})["history"]

# 메모리 초기화 함수
def reset_memory():
    memory.clear()

# 영화 제목에 대한 예시와 그에 해당하는 이모티콘 응답
examples = [
    {"input": "Top Gun", "output": "🛩️👨‍✈️🔥"},
    {"input": "The Godfather", "output": "👨‍👨‍👦🔫🍝"},
    {"input": "Finding Nemo", "output": "🐟🔍🌊"},
    {"input": "Jurassic Park", "output": "🦖🌴🚙"},
    {"input": "Harry Potter", "output": "🪄🧙‍♂️🏰"},
]

# 예시 메시지의 포맷을 정의
example_prompt = ChatPromptTemplate.from_messages(
    [('human', '{input}'), ('ai', '{output}')]
)

# FewShotChatMessagePromptTemplate을 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
)

# 최종 프롬프트를 작성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', 'You are a helpful AI Assistant that responds with three emojis for movie titles.'),
        few_shot_prompt,
        ('human', '{question}'),
    ]
)

# 모델을 ChatOpenAI로 설정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)

# 메모리 설정 (대화 요약 메모리 사용)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

# 메모리와 프롬프트 연결
chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

# 체인 호출 함수
def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result.content)




In [73]:
reset_memory()

# 첫 번째 영화에 대한 질문
invoke_chain("미니언즈")

# 두 번째 영화에 대한 질문
invoke_chain("기생충")


🍌😂💛
🏠🪰👨‍👩‍👧‍👦


In [78]:
# 메모리에서 첫 번째 영화가 무엇인지 확인
print(memory.load_memory_variables({})["history"][1])  # 첫 번째 질문의 기억

content='🍌😂💛'
